In [98]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [99]:
f1 = open("Sample input and output for HMM/Input/data.txt",'r')
f2 = open("Sample input and output for HMM/Input/parameters.txt.txt",'r')
classNames = ["El Nino", "La Nina"]
num_array = np.array([[float(num) for num in f1.read().split()]])
state_no = int((f2.readline().split())[0])
trans_mat1 = []
for i in range(state_no):
    tmp = [float(num) for num in f2.readline().split()]
    trans_mat1.append(tmp)
df = pd.DataFrame(num_array)
trans_mat1 = np.array(trans_mat1)
means1 = [float(num) for num in f2.readline().split()]
stds1 = [np.sqrt(float(num)) for num in f2.readline().split()]
means1,stds1,trans_mat1,num_array

([200.0, 100.0],
 [3.1622776601683795, 3.1622776601683795],
 array([[0.7, 0.3],
        [0.1, 0.9]]),
 array([[104.52431766,  98.29539094, 101.20130057,  96.82588248,
          99.77778448, 100.53588206, 150.21924799, 153.29917143,
         152.03045238, 149.27144663, 148.96539119, 153.2095234 ,
         148.53611628, 151.04160094, 152.80693308, 154.14613237,
         101.47489507,  99.41815223,  99.74916799,  97.38016976,
         150.70027155, 146.08854778, 152.14222891, 104.34605754,
         152.94106549, 147.63105659, 147.40077903, 145.46566737,
         152.76234462, 147.5848832 , 147.46342876, 150.19276708,
         103.72260951,  99.50572002, 154.25160489, 148.61671894,
         149.20219246, 152.51934586, 145.86757651, 153.74335161,
         103.92558835,  99.91585177,  99.48557241, 146.39467121,
         146.83909249,  99.84860756, 103.09450851, 105.01845328,
          94.62662942, 100.41078987, 100.78063234,  98.35851344,
         151.29390318, 100.97865142, 151.30078712, 15

In [100]:
def get_stationary_dist(mat):
    arr = mat.T[:-1]
    arr = np.r_[arr,[np.ones(arr.shape[1])]]
    for i in range(arr.shape[0]-1):
        for j in range(arr.shape[1]):
            if i==j:
                arr[i][j] -= 1
    b = np.zeros((arr.shape[0],1))
    b[b.shape[0]-1][0] = 1
    
    ans = np.linalg.solve(arr,b)
    return ans

st_ds = get_stationary_dist(trans_mat1).T
stat_dis = st_ds
#print(st_ds)
st_ds = np.array([np.log2(val) for val in st_ds])

#st_ds

In [101]:
def get_probs(num_array,means1,stds1,vit=1):
    probs = []
    for x in num_array[0]:
        vals = []
        for i in range(len(means1)):
            #val = 1/(stds1[i]*np.sqrt(2*np.pi))*np.exp(-.5*((x-means1[i])**2/stds1[i]))
            val = norm.pdf(x,loc=means1[i],scale=stds1[i])
            if(vit==1): val = np.log2(val)
            vals.append(val)
        
        probs.append(vals)
    probs = np.array(probs)
    return probs
probs = get_probs(num_array,means1,stds1)
#probs.shape

In [102]:
def get_class(st_ds,t_m,probs,classNames):
    classes = []
    t_m = np.array([np.log2(val) for val in t_m])
    for i in range(1,probs.shape[0]+1):
        temp = []
        cls = []
        for j in range(st_ds.shape[1]):
            mx = None
            idx = -1
            for k in range(st_ds.shape[1]):
                tmp = st_ds[0][k] + t_m[k][j] + probs[i-1][j]
                if mx == None:
                    mx = tmp
                    idx = k
                else :
                    if mx<tmp :
                        mx = tmp
                        idx = k
            cls.append(idx)
            temp.append(mx)
        st_ds = np.array([[float(val) for val in temp]])
        classes.append(cls)
    
    idx = None
    mx = None
    for i in range(st_ds.shape[1]):
        x = st_ds[0][i]
        if mx == None:
            mx = x
            idx = i
        else :
            if mx<x :
                mx = x
                idx = i
    ans = []
    ans.append(idx)
    for x in reversed (classes):
        if len(ans) == 0:
            ans.append(np.argmax(x))
        else : 
            ans.append(x[ans[len(ans)-1]])
        
    ans = ans[:-1]
    classes = []
    for x in reversed(ans):
        classes.append(classNames[x])
    return classes
clss = get_class(st_ds,trans_mat1,probs,classNames)
clss
f3 = open("Sample input and output for HMM/Output/states_Viterbi_wo_learning.txt",'r')
lines = [line.replace('"',"") for line in f3.read().splitlines()]
cnt = 0
print("Predicted","\t","Actual")
for i in range(len(clss)):
    print(clss[i],"\t",lines[i])
    if(clss[i]==lines[i]): 
        cnt += 1
        #print(1)
    #else: print(0)
print("accuracy : ",cnt/len(clss)*100,"%\n")

Predicted 	 Actual
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
El Nino 	 El Nino
El Nino 	 El Nino
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
El Nino 	 El Nino
El Nino 	 El Nino
El Nino 	 El Nino
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
La Nina 	 La Nina
El Nino 	 El Nino
La Nina 	 La Nina
El Nino 	

In [103]:
np.random.seed(150)
trans_mat1 = np.random.rand(trans_mat1.shape[0],trans_mat1.shape[1])
tr_mat = []
for i in range(trans_mat1.shape[0]) :
        tr_mat.append([val/np.sum(trans_mat1[i]) for val in trans_mat1[i]])
trans_mat1 = np.array(tr_mat)
means1 = np.random.rand(len(means1))*250
stds1 = np.random.rand(len(means1))*20
trans_mat1,means1,stds1

(array([[0.77886039, 0.22113961],
        [0.54289493, 0.45710507]]),
 array([174.51913013, 129.3021378 ]),
 array([19.0421926 , 18.27289039]))

In [104]:
def printarr(arr):
    for val in arr:
        print(val, end = " ")
    print()

In [107]:

def set_class(t_m,probs):
    
    
    forward = np.zeros((probs.shape[0], t_m.shape[0]))
    backward = np.zeros((probs.shape[0], t_m.shape[0]))
    pix = np.zeros((probs.shape[0], t_m.shape[0]))
    pixx = np.zeros((probs.shape[0]-1, t_m.shape[0]* t_m.shape[0]))
    st_ds = get_stationary_dist(trans_mat1).T
    #print(st_ds.shape)
    for i in range(probs.shape[0]):

        st_ds = (st_ds@t_m)*probs[i]
        #print(st_ds.shape)   
        st_ds = np.array([val/np.sum(st_ds) for val in st_ds])
        forward[i] = st_ds
    st_ds = np.ones((1,t_m.shape[0]))
    backward[probs.shape[0]-1] = st_ds
    for i in range(probs.shape[0]-2,-1,-1):


        st_ds = (t_m@(st_ds*probs[i+1]).T).T
           
        st_ds = np.array([val/np.sum(st_ds) for val in st_ds])
        backward[i] = st_ds
        

    #np.savetxt("forwardMatrix.txt",forward,delimiter=' ')
    #np.savetxt("backwardMatrix.txt",backward,delimiter=' ')
    #print(forward,backward)

    for i in range(probs.shape[0]):
        pix[i] = forward[i]*backward[i]
        pix[i] = np.array([val/np.sum(pix[i]) for val in pix[i]])
        if i>0:
            fr = np.array([forward[i-1]]).T
            #print("b",fr)
            fr =  (fr * t_m)
            #print("a",fr)
            pixx[i-1] = (fr*(backward[i]*probs[i])).flatten()
            pixx[i-1] = np.array([val/np.sum(pixx[i-1]) for val in pixx[i-1]])
    
    bb = np.sum(pixx,axis=0)
    #print(bb.shape)
    avg_probs = np.sum(pix,axis = 0)
    #print(avg_probs)
    prob_rains = np.sum(pix*(num_array.T),axis = 0)
    #print(prob_rains)
    avg_rains = prob_rains/avg_probs
    #print(avg_rains)
    std_probs = np.sum(pix*(num_array.T-avg_rains)**2,axis = 0)
    #print(std_probs)
    std_rains = np.sqrt(std_probs/avg_probs)
    #print(std_rains)
    n = (int)(np.sqrt(bb.shape[0]))
    ret = np.reshape(bb,(n,n))
    tr_mat = []
    for i in range(ret.shape[0]) :
        tr_mat.append([val/np.sum(ret[i]) for val in ret[i]])
    ret = np.array(tr_mat)
    #print(ret.shape)
    return ret, avg_rains, std_rains
i = 0
while i<10:
    print("Iteration :" , i+1)
    probs = get_probs(num_array,means1,stds1,vit=0)
    ret, avg_rains, std_rains = set_class(trans_mat1,probs)
    trans_mat1 = ret
    means1 = avg_rains
    stds1 = std_rains
    #print(ret, avg_rains, std_rains)
    for vals in trans_mat1:
        printarr(vals)
    printarr(means1)
    printarr(stds1)
    i += 1


probs = get_probs(num_array,means1,stds1)
print(probs)
st_ds = get_stationary_dist(trans_mat1).T
#print(st_ds)
clss = get_class(st_ds,trans_mat1,probs,classNames)
clss
f4 = open("Sample input and output for HMM/Output/states_Viterbi_after_learning.txt",'r')
lines = [line.replace('"',"") for line in f4.read().splitlines()]
cnt = 0
print("\nPredicted","\t","Actual")
for i in range(len(clss)):
    print(clss[i],"\t",lines[i])
    if(clss[i]==lines[i]): 
        cnt += 1
        #print(1)
    else: print(clss[i]," ",lines[i],0)
print("accuracy : ",cnt/len(clss)*100,"%")
print()
stds1 = [val*val for val in stds1]
for vals in trans_mat1:
    printarr(vals)
printarr(means1)
printarr(stds1)
for val in stat_dis:
    printarr(val)

Iteration : 1
0.8279570908782721 0.17204290912172807 
0.2176868598456676 0.7823131401543324 
150.1898696730972 100.20941303897588 
2.2431838527486425 2.951935568324583 
Iteration : 2
0.8279569892473119 0.17204301075268819 
0.21768707482993196 0.782312925170068 
150.1898689024755 100.20940295792063 
2.243184565819609 2.9518579138648846 
Iteration : 3
0.8279569892473119 0.17204301075268819 
0.21768707482993196 0.782312925170068 
150.1898689024755 100.20940295792063 
2.243184565819609 2.9518579138648846 
Iteration : 4
0.8279569892473119 0.17204301075268819 
0.21768707482993196 0.782312925170068 
150.1898689024755 100.20940295792063 
2.243184565819609 2.9518579138648846 
Iteration : 5
0.8279569892473119 0.17204301075268819 
0.21768707482993196 0.782312925170068 
150.1898689024755 100.20940295792063 
2.243184565819609 2.9518579138648846 
Iteration : 6
0.8279569892473119 0.17204301075268819 
0.21768707482993196 0.782312925170068 
150.1898689024755 100.20940295792063 
2.243184565819609 2.9518

In [106]:
'''f3 = open("Sample input and output for HMM/Output/states_Viterbi_after_learning.txt",'r')
lines = [line.replace('"',"") for line in f3.read().splitlines()]
cnt = 0
for i in range(len(clss)):
    print(clss[i]," ",lines[i],end=" ")
    if(clss[i]==lines[i]): 
        cnt += 1
        print(1)
    else: print(0)
#cnt = np.sum(clss==lines)
print("accuracy : ",cnt/len(clss)*100,"%")'''

'f3 = open("Sample input and output for HMM/Output/states_Viterbi_after_learning.txt",\'r\')\nlines = [line.replace(\'"\',"") for line in f3.read().splitlines()]\ncnt = 0\nfor i in range(len(clss)):\n    print(clss[i]," ",lines[i],end=" ")\n    if(clss[i]==lines[i]): \n        cnt += 1\n        print(1)\n    else: print(0)\n#cnt = np.sum(clss==lines)\nprint("accuracy : ",cnt/len(clss)*100,"%")'